# Tutorial 0: Debugging common error messages and issues in Python, Pandas and Biogeme

This tutorial covers some of the most common error messages and issues that may arise when working with Python and Biogeme for discrete choice modelling. We will go through typical mistakes, explain the error messages, and provide solutions to fix them.<br>
This tutorial comprises the following sections:
1. [Python errors](#1-python-errors)
    1. `ModuleNotFoundError`
    1. `ERROR: Could not find a version that satisfies the requirement`
2. [Pandas errors](#2.-pandas-errors)
    1. `FileNotFoundError:`
3. [Biogeme errors](#3.-biogeme-errors)
    1. `AttributeError: 'DataFrame' object has no attribute`
    1. `KeyError:`
    1. `BiogemeError: Error in the loglikelihood function.`
    1. `BiogemeError: The argument of MonteCarlo must contain a bioDraws.`
    1. `BiogemeError: The following elementary expressions are defined more than once.`

In [1]:
# Let's first import some native Python libraries
import sys
import os

## 1. Python errors

<br>

`ModuleNotFoundError: No module named 'utils'`

In [2]:
# We try to import a module from the utils folder
# To do so, we add the path where the folder utils can be found to the Python path using sys.path.append()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

# Import the bio_estimation_fcns from the utils folder
from utils.bio_estimation_fcns import print_results

ModuleNotFoundError: No module named 'utils'

In [3]:
# This error occurs because the folder utils is not found in the specified path.
# Let's print the path where Python is looking for the utils folder
print(f"Python looked for the folder named 'utils' in the directory:\n{os.path.abspath(os.path.join(os.getcwd(), '..', '..'))}")

# However, the utils folder is not in this folder
# It is NOT located TWO levels up from the current working directory os.getcwd(), but rather ONE level up.
# Note: '..' means go up one level in the directory tree. So, '..', '..' means go up two levels in the directory tree
print(f"\nThe correct path to look for the 'utils' folder should be:\n{os.path.abspath(os.path.join(os.getcwd(), '..'))}")

# Note: we use os.path.abspath() to ensure we have a clean absolute path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # <-- FIXED by adding the correct path

# Now we have added the correct path, we can import the bio_estimation_fcns from the utils folder
from utils.bio_estimation_fcns import print_results, estimate_mnl, estimate_mxl, estimate_panel_mxl

Python looked for the folder named 'utils' in the directory:
/Users/sandervancranenburgh/Documents/Repos_and_data/SEN1221

The correct path to look for the 'utils' folder should be:
/Users/sandervancranenburgh/Documents/Repos_and_data/SEN1221/tutorials


<br>

`ERROR: Could not find a version that satisfies the requirement`

In [4]:
pip install whatever-package

ERROR: Could not find a version that satisfies the requirement whatever-package (from versions: none)
ERROR: No matching distribution found for whatever-package
Note: you may need to restart the kernel to use updated packages.


In [5]:
# This error occurs because the Python interpreter cannot find the specified package in its search path.
# To fix this, you need to ensure that the package is installed in your Python environment.
# You can install the package using pip or conda, depending on your package manager.
# If you are using pip, you can run the following command in your terminal or command prompt:
# pip install whatever-package # <-- FIXED by installing the missing package

## 1. Pandas errors

In [6]:
# Import pandas
import pandas as pd
from pathlib import Path

<br>

`FileNotFoundError: [Errno 2] No such file or directory:`

In [7]:
# Load the choice data

# First we specify the data path
data_path =  Path(f'synthetic_VTTdata_tutorial1.dat')

# Then we load the data into a pandas DataFrame
df = pd.read_csv(data_path)

FileNotFoundError: [Errno 2] No such file or directory: 'synthetic_VTTdata_tutorial1.dat'

In [8]:
# This error occurs when we try to load the data, but the file does not exist in the specified path.
# To fix this, ensure that the path pointing to the data file is correct.
print(f"Python looked for the data file in the path:\n{data_path.resolve()}")

print(f"\nThe specified path is relative to the current working directory:\n{os.getcwd()}")

# However, the file is not located there. It is located in the 'data' folder which is located in the current working directory.
data_path =  Path(f'data/synthetic_VTTdata_tutorial1.dat') # <-- FIXED by specifying the correct path to the data file

# Now, we can load the data using the correct path
df = pd.read_csv(data_path)

# Show the data
df.head(5)

Python looked for the data file in the path:
/Users/sandervancranenburgh/Documents/Repos_and_data/SEN1221/tutorials/tutorial0/synthetic_VTTdata_tutorial1.dat

The specified path is relative to the current working directory:
/Users/sandervancranenburgh/Documents/Repos_and_data/SEN1221/tutorials/tutorial0


,RESP,TC1,TT1,TC2,TT2,CHOICE
0,1,4,35,8,25,2
1,1,4,35,8,30,2
2,1,6,40,7,20,2
3,1,5,30,6,25,2
4,1,5,40,7,20,1


## 3. Biogeme errors

In [9]:
# Import biogeme libraries
import biogeme.database as db
from biogeme.expressions import Beta, Variable, bioDraws

<br>

`AttributeError: 'DataFrame' object has no attribute 'data'`

In [10]:
# Give a name to the model    
model_name = 'Linear-additive RUM-MNL'

# We create Variable objects for each of the variables in the data set that we want to use in the model
TT1  = Variable('TT1')
TC1  = Variable('TC1')
TT2  = Variable('TT2')
TC2  = Variable('TC2')
CHOICE = Variable('CHOICE')

# Define the parameters to be estimated
ASC1 = Beta('ASC1', 0, None, None, 1)
ASC2 = Beta('ASC2', 0, None, None, 0)
B_TT = Beta('B_TT', 0, None, None, 0)
B_TC = Beta('B_TC', 0, None, None, 0)

# Define the utility functions
V1 = ASC1 + B_TT * TT1 + B_TC * TC1
V2 = ASC2 + B_TT * TT2 + B_TC * TC2

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V1, 2: V2}
    
# Create a dictionary called AV to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
AV = {1: 1, 2: 1} 

# Estimate the model
results_mnl = estimate_mnl(V, AV, CHOICE, df, model_name)


AttributeError: 'DataFrame' object has no attribute 'data'

In [11]:
# The error occurs because the function biogeme, which is invoked in estimate_mnl() expects a Biogeme database object. 
# But a pandas DataFrame is being passed instead.

# To fix this, you need to create a Biogeme database object from the pandas DataFrame before passing it to the biogeme function.
# You can do this by using the biogeme.database.Database class to create a database object
biodata = db.Database("my_data", df) # <-- FIXED by creating a Biogeme database object

# Let's print the type of the biodata object to confirm it is a Biogeme database object
print(f"The object 'biodata' has type {type(biodata)}")

# Now we can pass the 'biodata' object to the estimate_mnl() function
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

# And print the results
print_results(results_mnl)

The object 'biodata' has type <class 'biogeme.database.Database'>


Results for model Linear-additive RUM-MNL
Nbr of parameters:		3
Sample size:			1000
Excluded data:			0
Null log likelihood:		-693.1472
Final log likelihood:		-516.5406
Likelihood ratio test (null):		353.2132
Rho square (null):			0.255
Rho bar square (null):			0.25
Akaike Information Criterion:	1039.081
Bayesian Information Criterion:	1053.804

       Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC2  1.0996        0.2564         4.29           0.0
B_TC -0.3814        0.0584        -6.53           0.0
B_TT -0.0825        0.0138        -5.99           0.0


<br>

`KeyError:` 

In [19]:
# Create a Biogeme database object from the pandas DataFrame 
biodata = db.Database("my_data", df)

# Give a name to the model    
model_name = 'Linear-additive RUM-MNL'

# We create Variable objects for each of the variables in the data set that we want to use in the model
TT1  = Variable('TT1')
TC1  = Variable('TC1')
TT2  = Variable('TT2')
TC2  = Variable('TC2')
CHOICE = Variable('Chosen')

# Define the parameters to be estimated
ASC1 = Beta('ASC1', 0, None, None, 1)
ASC2 = Beta('ASC2', 0, None, None, 0)
B_TT = Beta('B_TT', 0, None, None, 0)
B_TC = Beta('B_TC', 0, None, None, 0)

# Define the utility functions
V1 = ASC1 + B_TT * TT1 + B_TC * TC1
V2 = ASC2 + B_TT * TT2 + B_TC * TC2

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V1, 2: V2}
    
# Create a dictionary called av to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
AV = {1: 1, 2: 1} 

# Estimate the model
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

KeyError: 'Chosen'

In [13]:
# This error occurs because the variable 'Chosen' does not exist in the Biogeme database object 'biodata'.
# To fix this, ensure that the variable name used in the Variable() function matches the name of the variable in the data set.

# We can print the column names of the DataFrame to check the exact variable names that do exist in the data set
print("Column names in the DataFrame:")
print(biodata.data.columns)

# We see that the correct variable name for the choice column is 'CHOICE', not 'Chosen'.
# So, we need to change the line where we create the Variable object for CHOICE
CHOICE = Variable('CHOICE') # <-- FIXED by using the correct variable name

# Now we can run the estimation again
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

# And print the results
print_results(results_mnl)

Column names in the DataFrame:
Index(['RESP', 'TC1', 'TT1', 'TC2', 'TT2', 'CHOICE'], dtype='object')


Results for model Linear-additive RUM-MNL
Nbr of parameters:		3
Sample size:			1000
Excluded data:			0
Null log likelihood:		-693.1472
Final log likelihood:		-516.5406
Likelihood ratio test (null):		353.2132
Rho square (null):			0.255
Rho bar square (null):			0.25
Akaike Information Criterion:	1039.081
Bayesian Information Criterion:	1053.804

       Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC2  1.0996        0.2564         4.29           0.0
B_TC -0.3814        0.0584        -6.53           0.0
B_TT -0.0825        0.0138        -5.99           0.0


<br>

`BiogemeError: Error in the loglikelihood function. Some variables are not inside PanelLikelihoodTrajectory:`

In [14]:
# The biogeme.database.Database.panel() method informs Biogeme that biodata is panel data, where 'RESP' is the variable in the data that identifies the individuals
# This method is necessary when estimating panel data models, such as panel mixed logit models.
biodata.panel('RESP')

# But, when we estimate a simple MNL model (which ignores the panel) is thrown an error.
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

BiogemeError: Error in the loglikelihood function. Some variables are not inside PanelLikelihoodTrajectory: {'TT1', 'CHOICE', 'TT2', 'TC2', 'TC1'} .If the database is organized as panel data, all variables must be used inside a PanelLikelihoodTrajectory. If it is not consistent with your model, generate a flat version of the data using the function `generateFlatPanelDataframe`.

In [15]:
# The error occurs because biogeme expects a model that accounts for the panel structure of the data: it looks for a PanelLikelihoodTrajectory use. 
# But the MNL does not account for panel structure.
# To fix this, you need to avoid providing information about the panel structure of the data when estimating a simple MNL model.
# So, you should not call the biodata.panel('RESP') line when estimating the MNL model.
# Alternatively, we can revert the biodata to a non-panel database using the following line:
biodata.panelColumn = None # <-- FIXED by reverting the biodata to a non-panel database

# Check if the biodata is indeed non-panel
print(biodata.is_panel())

# Now we can estimate the MNL model again
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

# And print the results
print_results(results_mnl)

False


Results for model Linear-additive RUM-MNL
Nbr of parameters:		3
Sample size:			1000
Excluded data:			0
Null log likelihood:		-693.1472
Final log likelihood:		-516.5406
Likelihood ratio test (null):		353.2132
Rho square (null):			0.255
Rho bar square (null):			0.25
Akaike Information Criterion:	1039.081
Bayesian Information Criterion:	1053.804

       Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC2  1.0996        0.2564         4.29           0.0
B_TC -0.3814        0.0584        -6.53           0.0
B_TT -0.0825        0.0138        -5.99           0.0


<br>

`BiogemeError: The argument of MonteCarlo must contain a bioDraws:`

In [16]:
# Give a name to the model    
model_name = 'Linear-additive Mixed Logit'

# Define the parameters to be estimated
ASC1 = Beta('ASC1', 0, None, None, 1)
ASC2 = Beta('ASC2', 0, None, None, 0)
B_TT = Beta('B_TT', 0, None, None, 0)
B_TC = Beta('B_TC', 0, None, None, 0)
sigma_TC = Beta('sigma_TC', 0.1, 0, None, 0)

# Define the random parameters
B_TC_rnd = B_TC + sigma_TC * bioDraws('B_TC_rnd', 'NORMAL')

# Define the utility functions
V1 = ASC1 + B_TT * TT1 + B_TC * TC1
V2 = ASC2 + B_TT * TT2 + B_TC * TC2

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V1, 2: V2}

# Create a dictionary called AV to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
AV = {1: 1, 2: 1} 

results_mxl = estimate_mxl(V, AV, CHOICE, biodata, model_name, num_draws=2000)

The argument of MonteCarlo must contain a bioDraws: MonteCarlo(exp(_bioLogLogit[choice=CHOICE]U=(1:((Beta('ASC1', 0, None, None, 1) + (Beta('B_TT', 0, None, None, 0) * TT1)) + (Beta('B_TC', 0, None, None, 0) * TC1)), 2:((Beta('ASC2', 0, None, None, 0) + (Beta('B_TT', 0, None, None, 0) * TT2)) + (Beta('B_TC', 0, None, None, 0) * TC2)))av=(1:`1.0`, 2:`1.0`)))


BiogemeError: The argument of MonteCarlo must contain a bioDraws: MonteCarlo(exp(_bioLogLogit[choice=CHOICE]U=(1:((Beta('ASC1', 0, None, None, 1) + (Beta('B_TT', 0, None, None, 0) * TT1)) + (Beta('B_TC', 0, None, None, 0) * TC1)), 2:((Beta('ASC2', 0, None, None, 0) + (Beta('B_TT', 0, None, None, 0) * TT2)) + (Beta('B_TC', 0, None, None, 0) * TC2)))av=(1:`1.0`, 2:`1.0`)))

In [17]:
# This error occurs because the random parameter B_TC_rnd is defined but not used in the utility functions.
# To fix this, you need to use the random parameter B_TC_rnd in the utility.

# Give a name to the model    
model_name = 'Linear-additive Mixed Logit'

# Define the parameters to be estimated
ASC1 = Beta('ASC1', 0, None, None, 1)
ASC2 = Beta('ASC2', 0, None, None, 0)
B_TT = Beta('B_TT', 0, None, None, 0)
B_TC = Beta('B_TC', 0, None, None, 0)
sigma_TC = Beta('sigma_TC', 0.1, 0, None, 0)

# Define the random parameters
B_TC_rnd = B_TC + sigma_TC * bioDraws('B_TC_rnd', 'NORMAL')

# Define the utility functions
V1 = ASC1 + B_TT * TT1 + B_TC_rnd * TC1 # <-- FIXED by using B_TC_rnd here
V2 = ASC2 + B_TT * TT2 + B_TC_rnd * TC2 # <-- FIXED by using B_TC_rnd here

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V1, 2: V2}
    
# Create a dictionary called av to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
AV =  {1: 1, 2: 1} 

# Estimate the model
results_mnl = estimate_mxl(V, AV, CHOICE, biodata, model_name, num_draws=2000)

# Show the results
print_results(results_mnl)



Number of estimated parameters:	4
Sample size:	1000
Excluded observations:	0
Init log likelihood:	-693.109
Final log likelihood:	-515.2759
Likelihood ratio test for the init. model:	355.6663
Rho-square for the init. model:	0.257
Rho-square-bar for the init. model:	0.251
Akaike Information Criterion:	1038.552
Bayesian Information Criterion:	1058.183
Final gradient norm:	2.5016E-03
Number of draws:	2000
Draws generation time:	0:00:01.540477
Types of draws:	['B_TC_rnd: NORMAL']
Nbr of threads:	10

           Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC2      0.9820        0.2909         3.38          0.00
B_TC     -0.3457        0.0935        -3.70          0.00
B_TT     -0.1143        0.0289        -3.95          0.00
sigma_TC  0.7843        0.4060         1.93          0.05


<br>

`BiogemeError: The following elementary expressions are defined more than once:`

In [18]:
# Create a Biogeme database object from the pandas DataFrame 
biodata = db.Database("my_data", df)

# Give a name to the model    
model_name = 'Linear-additive RUM-MNL'

# We create Variable objects for each of the variables in the data set that we want to use in the model
TT1  = Variable('TT1')
TC1  = Variable('TC1')
TT2  = Variable('TT2')
TC2  = Variable('TC2')
CHOICE = Variable('CHOICE')

# Define the parameters to be estimated
ASC1 = Beta('ASC', 0, None, None, 1) # <-- Here is the cause of the error
ASC2 = Beta('ASC', 0, None, None, 0) # <-- Here is the cause of the error
B_TT = Beta('B_TT', 0, None, None, 0)
B_TC = Beta('B_TC', 0, None, None, 0)

# Define the utility functions
V1 = ASC1 + B_TT * TT1 + B_TC * TC1
V2 = ASC2 + B_TT * TT2 + B_TC * TC2

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V1, 2: V2}
    
# Create a dictionary called av to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
av = {1: 1, 2: 1} 

# Estimate the model
results_mnl = estimate_mnl(V, AV, CHOICE, biodata, model_name)

BiogemeError: The following elementary expressions are defined more than once: {'ASC'}.

In [ ]:
# The error occurs because the ASC parameters ASC1 and ASC2 are defined with the same name: just 'ASC'.
# In Biogeme, each parameter must have a unique name.

# This fix this, we need TO RESTART the KERNEL.
# This clears the previous definitions. After the restart we can redefine the ASC parameters with unique names:

# ASC1 = Beta('ASC1', 0, None, None, 1) # <-- FIXED by using unique parameter names
# ASC2 = Beta('ASC2', 0, None, None, 0) # <-- FIXED by using unique parameter names